In [1]:
from transforna import GeneEmbeddModel,RnaTokenizer
import torch
model_name = 'Seq-Struct'
model_path = f"Yak-hbdx/{model_name}-TransfoRNA"

In [2]:
#load model and tokenizer
model = GeneEmbeddModel.from_pretrained(model_path)
model.eval()

tokenizer = RnaTokenizer.from_pretrained(model_path,model_name=model_name)
output = tokenizer(['AAAGTCGGAGGTTCGAAGACGATCAGATAC','TTTTCGGAACTGAGGCCATGATTAAGAGGG'])

gene_embedd, second_input_embedd, activations,attn_scores_first,attn_scores_second = \
                                model(output['input_ids'])
#get the idx of the maximum value in the gene_embedd tensor at each row
class_labels = model.convert_ids_to_labels(activations)
#asset ['18S_bin-38', '18S_bin-33']
assert class_labels == ['18S_bin-38', '18S_bin-33'], print('\033[91m' + 'FAILED')
print('\033[92m' + 'PASSED')

Tokenizing sequences for seq-struct model...
PASSED
